In [49]:
%load_ext tensorboard
import datetime, os
import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

num_of_classes = ds_info.features['label'].num_classes
image_shape = ds_info.features['image'].shape
train_size = ds_info.splits['train'].num_examples
test_size = ds_info.splits['test'].num_examples


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [50]:
id_number = "206812042"

neurons_layer_1 = int(id_number[-2:])
neurons_layer_2 = int(id_number[-4:-2])
neurons_layer_3 = int(id_number[-6:-4])
neurons_layer_4 = int(id_number[-8:-6])

print(f"Number of neurons per layer: {neurons_layer_1}, {neurons_layer_2}, {neurons_layer_3}, {neurons_layer_4}")

Number of neurons per layer: 42, 20, 81, 6


In [51]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)


ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [52]:
layers = [
    tf.keras.layers.Flatten(input_shape=image_shape),

    tf.keras.layers.Dense(neurons_layer_1),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),

    tf.keras.layers.Dense(neurons_layer_2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),

    tf.keras.layers.Dense(neurons_layer_3),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('tanh'),

    tf.keras.layers.Dense(neurons_layer_4),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('tanh'),

    tf.keras.layers.Dense(num_of_classes),
    tf.keras.layers.Softmax()
]

In [53]:
model = tf.keras.models.Sequential(layers)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_8 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_40 (Dense)                     │ (None, 42)                  │          32,970 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_32               │ (None, 42)                  │             168 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_32 (Activation)           │ (None, 42)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 20)                  │             860 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_33               │ (None, 20)                  │              80 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_33 (Activation)           │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_42 (Dense)                     │ (None, 81)                  │           1,701 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_34               │ (None, 81)                  │             324 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_34 (Activation)           │ (None, 81)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 6)                   │             492 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_35               │ (None, 6)                   │              24 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_35 (Activation)           │ (None, 6)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_44 (Dense)                     │ (None, 10)                  │              70 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_8 (Softmax)                  │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 36,689 (143.32 KB)

 Trainable params: 36,391 (142.15 KB)

 Non-trainable params: 298 (1.16 KB)

In [54]:
model.fit(ds_train,
          epochs=50,
          validation_data=ds_test)

Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - loss: 0.9681 - sparse_categorical_accuracy: 0.7986 - val_loss: 0.2456 - val_sparse_categorical_accuracy: 0.9424
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.2847 - sparse_categorical_accuracy: 0.9255 - val_loss: 0.1677 - val_sparse_categorical_accuracy: 0.9558
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.2186 - sparse_categorical_accuracy: 0.9418 - val_loss: 0.1475 - val_sparse_categorical_accuracy: 0.9611
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 0.1846 - sparse_categorical_accuracy: 0.9486 - val_loss: 0.1302 - val_sparse_categorical_accuracy: 0.9659
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.1721 - sparse_categorical_accuracy: 0.9512 - val_loss: 0.1230 - val_sparse_categorical_accuracy: 0.9684
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.1536 - sparse_categorical_accuracy: 0.9557 - val_loss: 0.1187 - val_sparse_categorical_accura